In [ ]:
from glob import glob
from langchain.chains import StuffDocumentsChain, RetrievalQA, LLMChain, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chat_models.base import BaseChatModel
from langchain.document_loaders import TextLoader
from langchain.embeddings import OpenAIEmbeddings, OllamaEmbeddings
from langchain.llms import Ollama, BaseLLM
from langchain.llms.base import LLM
from langchain.schema import ChatResult, ChatGeneration
from langchain.schema import Document, Generation, LLMResult
from langchain.schema.messages import AIMessage, HumanMessage
from langchain.vectorstores import Chroma
from langchain_chroma import Chroma
from langchain_community.llms import OpenAI
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain.schema import OutputParserException
from pathlib import Path
from pydantic import BaseModel, Field
from tqdm import tqdm
from typing import List, Optional, Any
import json
import numpy as np
import pandas as pd
import requests

MODEL="llama3.3"
API_URL="http://127.0.0.1:11434"
FILE="echantillon_1000_hs_2024_TOC.parquet"
QUESTIONS=["De combien est le contingent annuel d'heures supplémentaires ?",
           "Quels sont les taux de majorations des heures supplémentaires ?" ,
           "Quel est le taux pour la contrepartie obligatoire en repos ?",
           "Quel est le taux pour le repos compensateur de remplacement ?",
          "Quel est le délai de prévenance pour les heures supplémentaires ?"]

OUTPUT_DIR_RESULTS="results"


In [ ]:
class ChatOllama(BaseChatModel):
    model: str = MODEL
    base_url: str = "http://localhost:11434"
    
    def _generate(self, messages: List[HumanMessage], stop: Optional[List[str]] = None, **kwargs) -> ChatResult:
        # Combine the messages into a single prompt
        prompt = "\n".join([f"{msg.content}" for msg in messages])
        response = requests.post(
            f"{self.base_url}/api/chat",
            json={"model": self.model,
  "temperature": 0, "messages": [{"role": "user", "content": prompt}], "stream": False},
        )
        response.raise_for_status()
        content = response.json()["message"]["content"]
        return ChatResult(generations=[ChatGeneration(message=AIMessage(content=content))])
    
    @property
    def _llm_type(self) -> str:
        return "ollama-chat"

llm = ChatOllama(api_url=API_URL)

class DonneesAccordsHeuresSupp(BaseModel):
    """Données structurées des accords d'heures supp"""
    index: Optional[str] = Field(default=None, description="Unique index")
    base_legale_hebdomadaire: Optional[int] = Field(
        default=None, description="Base légale hebdomadaire, 35 heures"
    )    
    duree_annuel_heures: Optional[int] = Field(
        default=None, description="Durée annuelle d'heures de travail, 1607 heures"
    )    
    contingent_annuel_heures_supplementaires: Optional[int] = Field(
        default=None, description="Nombre d'heures au contingent annuel d'heures supplémentaires"
    )
    nombre_taux_majoration_differents: Optional[int] = Field(
        default=None, description="Nombre de taux de majoration différents des heures supplémentaires en heures supplémentaires payées (hors contrepartie obligatoire en repos et repos compensateur de remplacement)"
    )
    premier_taux_majoration: Optional[int] = Field(
        default=None, description="Premier taux de majoration"
    )
    plage_premier_taux_majoration: Optional[str] = Field(
        default=None, description="Plage des heures du premier taux de majoration"
    )
    deuxieme_taux_majoration: Optional[int] = Field(
        default=None, description="Deuxième taux de majoration"
    )

    plage_deuxieme_taux_majoration: Optional[str] = Field(
        default=None, description="Plage des heures du deuxième taux de majoration"
    )
    troisieme_taux_majoration: Optional[int] = Field(
        default=None, description="Troisième taux de majoration"
    )
    plage_troisieme_taux_majoration: Optional[str] = Field(
        default=None, description="Plage des heures du troisième taux de majoration"
    )
    presence_repos_compensateur_remplacement: Optional[bool] = Field(
        default=None, description="Mention à un repos compensateur de remplacement (RCR)"
    )
    taux_majoration_contrepartie_obligatoire_en_repos: Optional[int] = Field(
        default=None, description="Taux de majoration de la contrepartie obligatoire en repos (COR)"
    )
    delai_prevenance: Optional[str] = Field(
        default=None, description="Délai de prévenance des heures supplémentaires"
    )
    
    
parser = PydanticOutputParser(pydantic_object=DonneesAccordsHeuresSupp)

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            (
                "You are an expert in extracting structured data.\n"
                "You MUST return only a JSON object in this exact format:\n\n"
                "{format_instructions}\n\n"
                "Do not include any other fields or text. No explanations. No markdown. Just valid JSON."
            ),
        ),
        ("human", "{query}"),
    ]
).partial(format_instructions=parser.get_format_instructions())


intermediate_chain = prompt | llm

    
embedder = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")
vector_store = Chroma(embedding_function=embedder, persist_directory="./chroma_db")


df=pd.read_parquet(FILE)
df=df.set_index("numdossier_new")

In [ ]:
DEBUG=False

Path(OUTPUT_DIR_RESULTS).mkdir(parents=True, exist_ok=True)
data=[]
for index, row in df.iterrows():
    retriever=vector_store.as_retriever(
        search_kwargs={
                "k": 2, 
                "filter": {'index': index}
            }
        )

    docs=[]
    for question in QUESTIONS:
        docs+= retriever.invoke(question)
    context="\n".join({d.page_content for d in docs})
    
    query = f"""
    Quelles sont les données de l'accord suivant ? 

    ### Données
    
    index={index}
    {context}
    """


    raw_output = intermediate_chain.invoke({"query": query})
    try:
        donnees_accords = parser.parse(raw_output.content)
        #print("Parsed output:", donnees_accords)
    except OutputParserException as e:
        donnees_accords = DonneesAccordsHeuresSupp()
        print("Index:\n", index)
        if DEBUG:
            print("Raw LLM output:\n", raw_output)
            print("Parsing failed:", e)
        
    try:
        donnees_accords.index=index
        with open(f"{OUTPUT_DIR_RESULTS}/heures_supp_{index}.json", "w", encoding="utf-8") as f:
            json.dump(donnees_accords.model_dump(), f, indent=2, ensure_ascii=False)
        data.append(donnees_accords)
    except:
        print(f"problème avec {index}")
df = pd.DataFrame([item.dict() for item in data])
df.to_parquet("results_heures_supp.parquet")